<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark_streaming/examples/1-read_write_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read & Write Stream
- readStream()
- writeStream()

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()

# readStream with format "rate"

In [2]:
import pyspark.sql.functions as F

# read stream
stream = spark.readStream.format("rate").load()



In [3]:
type(stream)

pyspark.sql.dataframe.DataFrame

In [5]:
# checking if it's streaming dataframe
stream.isStreaming

True

In [7]:
# should be false
data = [("c1", "v1"), ("c2", "v2")]
columns = ["col1", "col2"]
df = spark.createDataFrame(data, columns)
df.isStreaming

False

In [8]:
# apply normal dataframe operations
stream.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



In [ ]:
# Queries with streaming sources must be executed with writeStream.start();
# stream.show()
stream.count()

# Transfor streaming dataframe

In [12]:
transformed = stream.withColumn("value2", F.col("value") * 2)

In [13]:
transformed.isStreaming

True

# write streaming dataframe - format memory
- writeStream
- format("memory")
- queryName
- outputMode
- start

In [15]:
query = (transformed.writeStream
  .format('memory')
  .queryName('rate_report')
  .outputMode('append')
  .start()
)

IllegalArgumentException: Cannot start query with name rate_report as a query with that name is already active in this SparkSession

In [ ]:
# transform
transformed = stream.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
)

In [ ]:
query.stop()